# azenta_30_1018285327 - rna_seq
This notebook will create all the necessary files, scripts and folders to pre-process the aforementioned project. Is designed to be used in a jupyter server deployed in a system running SLURM. The majority of the scripts and heavy-lifting processes are wrapped up in sbatch scripts.As an end user, in order to pre-process your samples provided in the spread sheet, you will simply need to *run the entire notebook* (Cell > Run all) and the system should take care of the rest for you.
#### Create necessary folder(s)

In [5]:
%%bash
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/raw_reads
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/processed_raw_reads
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/jsons
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs

Save metadata file

In [6]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata/rna_seq_download_metadata.azenta_30_1018285327.txt
Sequencing core user	Sequencing core project	Sequencing core password	Sequencing core library name	Name	Paired-end or single-end	Genome	Library type	Strand specificity	With ercc spike-in
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-1Kpa-B1	A549.rnaseq.1kPa.rep1	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-1Kpa-B2	A549.rnaseq.1kPa.rep2	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-1Kpa-B3	A549.rnaseq.1kPa.rep3	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-50Kpa-B1	A549.rnaseq.50kPa.rep1	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-50Kpa-B2	A549.rnaseq.50kPa.rep2	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-50Kpa-B3	A549.rnaseq.50kPa.rep3	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-TCP-B1	A549.rnaseq.TCP.rep1	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-TCP-B2	A549.rnaseq.TCP.rep2	PE	hg38	RNA-seq	unstranded	False
alexias_safi_duke	30-1018285327	DTwykexD5cDYvpOJ7zxS	A549-TCP-B3	A549.rnaseq.TCP.rep3	PE	hg38	RNA-seq	unstranded	False


Overwriting /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata/rna_seq_download_metadata.azenta_30_1018285327.txt


#### Download FASTQ from sftp
Create file to download FASTQ files

In [9]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/download_azenta_30_1018285327.sh
#!/bin/bash
METADATA=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata/rna_seq_download_metadata.azenta_30_1018285327.txt
DATA_HOME=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq
mkdir -p ${DATA_HOME}/raw_reads/

read CORE_USER CORE_PROJECT CORE_PASS <<< $(cut -f 1,2,3 $METADATA | tail -n+2 | sort -u)
cd ${DATA_HOME}/raw_reads/
lftp -u ${CORE_USER},${CORE_PASS} -e "mirror --parallel=16 --only-missing ${CORE_PROJECT}; exit" "sftp://${CORE_USER}@sftp.genewiz.com"
exit


Overwriting /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/download_azenta_30_1018285327.sh


Execute file to download files

In [21]:
%%script --out blocking_job_str bash
sbatch -o /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs/azenta_30_1018285327_download_fastq_files.out \
 -p all \
 --wrap="ssh aeb84@Hardac-xfer.genome.duke.edu 'sh /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/download_azenta_30_1018285327.sh' "

Extract blocking job id

In [22]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Merge lanes of FASTQ files

In [33]:
%%bash
# quick symlink fix 
ln -s /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/raw_reads/30-1018285327/00_fastq/ \
    /data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/raw_reads/azenta_30_1018285327

In [23]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/merge_lanes_azenta_30_1018285327.sh
#!/bin/bash
#SBATCH --array=0-9%20
ORDER=azenta_30_1018285327
RAW_DATA_DIR=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/raw_reads/${ORDER}
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata/rna_seq_download_metadata.azenta_30_1018285327.txt

mkdir -p ${PROCESSED_DATA_DIR}
cd ${PROCESSED_DATA_DIR}

seq_name_header=$(/bin/grep -Eoi "sequencing.?core.?library.?name" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Sequencing core library name not found in ${METADATA}"
    exit 1
fi

name_header=$(/bin/grep -Poi "\tname\t" ${METADATA})
if [[ $? == 1 ]];
then
    echo -e "ERROR: Library Name column not found in ${METADATA}"
    exit 1
fi
name_header=$(echo ${name_header} | cut -f2)

seq_type_header=$(head -1 ${METADATA} | /bin/grep -Poi "paired.?end.?or.?single.?end")
if [[ $? == 1 ]];
then
    echo -e "ERROR: Paired-end or single-end column not found in ${METADATA}"
    exit 1
fi

sample_seq_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
sample_name=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${name_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');
seq_type=$(/data/reddylab/software/bin/print_tab_cols.awk -v cols="${seq_type_header}" ${METADATA} \
    | awk -v SLURM_ARRAY_TASK_ID=${SLURM_ARRAY_TASK_ID} 'NR==SLURM_ARRAY_TASK_ID+1{print}');


for read_pair in R1 R2 UMI;
do
    sample_files=$(/bin/ls ${RAW_DATA_DIR}/${sample_seq_name/ /}_${read_pair}_[0-9][0-9][0-9]*fastq.gz 2> /dev/null)
    if [[ $? != 0 ]]; # If no samples found with that read_pair, continue
    then
        continue;
    fi
    if [[ ${read_pair} == "R1" || (${seq_type/ /} == "PE" || ${seq_type/ /} == "pe") ]];
    then
        # Merge all lanes
        merged=$(basename $(echo ${sample_files} | awk '{print $1}') | sed -e 's/_[0-9]\{3\}\./_/')
        cat ${sample_files} > ${merged};

        # Rename samples with our sample Names
        dest_filename=$(basename $(echo ${merged} | awk '{print $1}') | sed -r 's/\_(R1|R2|UMI)\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
        mv ${merged} ${dest_filename}

        cleaned_dest_filename=${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

        if [[ ${seq_type/ /} == "SE" || ${seq_type/ /} == "se" ]];
        then
            cleaned_dest_filename=${cleaned_dest_filename/.R1/}
        fi
        
        mv ${dest_filename} ${cleaned_dest_filename}
    fi
done


Overwriting /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/merge_lanes_azenta_30_1018285327.sh


In [19]:
%%bash
RAW_DATA_DIR='/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/raw_reads/30-1018285327/00_fastq'
sample_seq_name='A549-1Kpa-B1'
sample_name='A549.rnaseq.1kPa.rep1'
read_pair='R1'
sample_files=$(/bin/ls ${RAW_DATA_DIR}/${sample_seq_name/ /}_${read_pair}_[0-9][0-9][0-9]*fastq.gz 2> /dev/null)
merged=$(basename $(echo ${sample_files} | awk '{print $1}') | sed -e 's/_[0-9]\{3\}\./_/')
dest_filename=$(basename $(echo ${merged} | awk '{print $1}') | sed -r 's/\_(R1|R2|UMI)\_/\.\1\./; s/\.[0-9]+\.fastq/\.fastq/')
echo ${dest_filename/${sample_seq_name/ /}/${sample_name/ /}}

A549.rnaseq.1kPa.rep1.R1.fastq.gz


Execute file to merge lanes of FASTQ files

In [34]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -o /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs/azenta_30_1018285327_merge_fastq_files_%a.out \
 -p all \
 --array 0-8%20 \
 /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/merge_lanes_azenta_30_1018285327.sh

Extract blocking job id

In [35]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create JSON files for CWL pipeline files

In [36]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/cwl_json_gen_azenta_30_1018285327.sh
#!/bin/bash
ORDER=azenta_30_1018285327
PROCESSED_DATA_DIR=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/processed_raw_reads/${ORDER}
METADATA=/data/reddylab/Alex/collab/20240529_Crawford/data/rna_seq/metadata/rna_seq_download_metadata.azenta_30_1018285327.txt

python /data/reddylab/software/cwl/GGR-cwl/v1.0/json-generator/run.py \
    -m ${METADATA} \
    -d ${PROCESSED_DATA_DIR} \
    -o /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/jsons \
    -t rna-seq \
    --fastq-gzipped \
    --mem 48000 \
    --nthreads 24 \
    --separate-jsons \
    --skip-star-2pass \


Overwriting /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/cwl_json_gen_azenta_30_1018285327.sh


Execute file to create JSON files

In [37]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -o /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs/azenta_30_1018285327_cwl_json_gen.out \
 -p all \
 --depend afterok:$1 \
 /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/cwl_json_gen_azenta_30_1018285327.sh

Extract blocking job id

In [38]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create SLURM array master bash file for pe-unstranded-with-sjdb samples

In [39]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/azenta_30_1018285327-pe-unstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=cwl_rna_seq
#SBATCH --output=/data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs/azenta_30_1018285327-pe-unstranded-with-sjdb-%a.out
#SBATCH --mail-user=aeb84@duke.edu
#SBATCH --mail-type=FAIL,END
#SBATCH --mem=48000
#SBATCH --cpus-per-task=24

export PATH="/data/reddylab/software/bin:$PATH"
export PATH="/data/common/shared_conda_envs/ucsc/bin:$PATH"
export PATH="/data/reddylab/software/cwl/bin:$PATH"
export PATH="/data/reddylab/software/preseq_v2.0:$PATH"
export PATH="/data/reddylab/software/rsem-1.2.21/:$PATH"
export PATH="/data/reddylab/software/STAR-STAR_2.4.1a/bin/Linux_x86_64/:$PATH"
export PATH="/data/reddylab/software/subread-1.4.6-p4-Linux-x86_64/bin/:$PATH"
export PATH="/data/reddylab/software/bamtools-2.2.3/bin/:$PATH"

export PATH="/data/reddylab/software/miniconda2/envs/cwl10/bin:$PATH"

module load bedtools2
module load fastqc
module load samtools
module load bowtie2
module load java

# For Fastqc
export DISPLAY=:0.0

# Make sure temporary files and folders are created in a specific folder
mkdir -p /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/tmpdirs/tmp-azenta_30_1018285327-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-
export TMPDIR="/data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/tmpdirs/tmp-azenta_30_1018285327-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-"

cwltool --debug \
    --non-strict \
    --preserve-environment PATH \
    --preserve-environment DISPLAY \
    --preserve-environment TMPDIR \
    --outdir /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/azenta_30_1018285327-pe-unstranded-with-sjdb  \
    --no-container \
    /data/reddylab/software/cwl/GGR-cwl/v1.0/RNA-seq_pipeline/pipeline-pe-unstranded-with-sjdb.cwl \
    /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/jsons/rna_seq_download_metadata.azenta_30_1018285327-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}.json

# Delete any tmpdir not removed by cwltool
rm -rf /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/tmpdirs/tmp-azenta_30_1018285327-pe-unstranded-with-sjdb-${SLURM_ARRAY_TASK_ID}-


Overwriting /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/azenta_30_1018285327-pe-unstranded-with-sjdb.sh


Execute SLURM array master file

In [40]:
%%script --out blocking_job_str bash -s "$blocking_job"
source /data/reddylab/software/miniconda2/bin/activate cwl10
sbatch -p all \
 --depend afterok:$1 \
 --array 0-8%20 \
 /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/azenta_30_1018285327-pe-unstranded-with-sjdb.sh

Extract blocking job id

In [41]:
import re
blocking_job = re.match('Submitted batch job (\d+).*', blocking_job_str).group(1)

#### Create QC generating script

In [42]:
%%writefile /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/generate_qc_cell_azenta_30_1018285327-pe-unstranded-with-sjdb.sh
#!/bin/bash
#SBATCH --job-name=qc
#SBATCH --output=/data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/logs/qc_gen.azenta_30_1018285327-pe-unstranded-with-sjdb.out

source /data/reddylab/software/miniconda2/bin/activate alex
cd /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/azenta_30_1018285327-pe-unstranded-with-sjdb

python /data/reddylab/software/cwl/bin/generate_stats_rnaseq_paired_end.py ./ \
    -samples $(/bin/ls -1 *PBC.txt | sed 's@.PBC.txt@@') \
> qc.txt

Overwriting /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/generate_qc_cell_azenta_30_1018285327-pe-unstranded-with-sjdb.sh


Generate QCs for azenta_30_1018285327-pe-unstranded-with-sjdb

In [ ]:
%%script --out blocking_job_str bash -s "$blocking_job"
sbatch -p all \
 --depend afterok:$1 \
 /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/scripts/generate_qc_cell_azenta_30_1018285327-pe-unstranded-with-sjdb.sh

Merge `featureCounts` files and add Gene Symbols

In [1]:
import pandas as pd
import re
import os
data_dir = '/data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/azenta_30_1018285327-pe-unstranded-with-sjdb'
comp_files = [
    [
        "A549.rnaseq.1kPa.rep1.star2.featurecounts.counts.txt",
        "A549.rnaseq.1kPa.rep2.star2.featurecounts.counts.txt",
        "A549.rnaseq.1kPa.rep3.star2.featurecounts.counts.txt",
        "A549.rnaseq.50kPa.rep1.star2.featurecounts.counts.txt",
        "A549.rnaseq.50kPa.rep2.star2.featurecounts.counts.txt",
        "A549.rnaseq.50kPa.rep3.star2.featurecounts.counts.txt",
        "A549.rnaseq.TCP.rep1.star2.featurecounts.counts.txt",
        "A549.rnaseq.TCP.rep2.star2.featurecounts.counts.txt",
        "A549.rnaseq.TCP.rep3.star2.featurecounts.counts.txt"
    ]
]

def clean_count_column(s):
    return ".".join(s.split('/')[-1].split('.')[:4])


for comp_i0, files in enumerate(comp_files):
    df = None
    comp_i1 = comp_i0 + 1

    for f in files:
        df_tmp = pd.read_csv(os.path.join(data_dir, f), sep='\t', skiprows=1, index_col=0)
        df_tmp.columns = df_tmp.columns[:-1].tolist() + [clean_count_column(f)]
        if df is None:
            df = df_tmp.loc[:, ['Chr', 'Start', 'End'] + [df_tmp.columns[-1]]]
        else:
            df = df.join(df_tmp.loc[:, df_tmp.columns[-1]], how='right')

    count_columns = ["rep" in c for c in df.columns]
    df.loc[:, count_columns] = df.loc[:, count_columns].round().astype(dtype='int')

    df.columns = [c.replace("-", "_") for c in df.columns]
    df.to_csv(f"{data_dir}/A549.rnaseq.featureCounts.all.txt",
             sep='\t')

    

In [ ]:
%%bash
source /data/reddylab/Alex/software/miniforge3/bin/activate intervene
add_gene_symbol_to_df.py \
    -ifile \
        /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/azenta_30_1018285327-pe-unstranded-with-sjdb/A549.rnaseq.featureCounts.all.txt \
    -outfile \
        /data/reddylab/Alex/collab/20240529_Crawford/processing/rna_seq/azenta_30_1018285327-pe-unstranded-with-sjdb/A549.rnaseq.featureCounts.all.with_gene_symbols.txt \
    -gene-id-to-symbol-format JSON \
    --gene-id-to-symbol-ix-id 0 \
    -gene-id-to-symbol \
        /data/reddylab/Reference_Data/Gencode/v22/gencode.v22.gene_id_to_gene_name.json